# Test technique pour le stage data science chez Bouygues Telecom

<div style="text-align: right;"><span style="font-style: italic; margin-right: 1.5em;">Gabriel Watkinson</span></div> 
<hr class="solid">

Ce notebook contient le test technique pour le stage de data science chez Bouygues Telecom basée sur des données factices sur la résiliation des clients ayant un abonnement télécom.

### Description du test

L'objectif est de comprendre et de prédire la résilation des clients. Pour ce faire 4 datasets sont fournis :

* resiliation_option : contient les données sur les options des forfaits fixe et mobile des clients
* resiliation_client : contient des données clients
* resiliation_contrat : contient les caractéristiques des contrats des clients
* resiliation_forfait : contient les caractéristiques du forfait mobile du client

### Organisation du notebook

Le notebook est composé de 3 parties :

1. Analyses des données et statistiques descriptives
2. Préparation de données
3. Modélisation du problème

## Analyses des données et statistiques descriptives

Dans un premier temps, nous allons analyser les données pour comprendre leur contenu et leur structure en utilisant des statistiques descriptives et des graphiques pour visualiser les distributions des données, ainsi que des problèmes éventuels, tels que les données manquantes par exemple, qu'il faudra pendre en compte lors de la modélisation.

### EDA automatique avec `pandas_profiling`

Afin de découvrir les données, nous allons utiliser `pandas_profiling` qui permet d'explorer les données et de soulever certain problèmes de manière automatique.

In [ ]:
import pandas as pd

# Chargement des données situées dans le dossier `data/`
data_folder = "./data/"
client = pd.read_csv(data_folder + "resiliation_client.csv", sep=";")
contrat = pd.read_csv(data_folder + "resiliation_contrat.csv", sep=";")
forfait = pd.read_csv(data_folder + "resiliation_forfait.csv", sep=";")
option = pd.read_csv(data_folder + "resiliation_option.csv", sep=";")

In [ ]:
import os

from pandas_profiling import ProfileReport

# Création des profiles automatiques
# Création du dossier `profiles` qui contient les profiles en html
os.makedirs("profils", exist_ok=True)

print("Pour la table client :")
profil_client = ProfileReport(client, title="Profil de la table client")
profil_client.to_file("profils/profil_client.html")

print("Pour la table contrat :")
profil_contrat = ProfileReport(contrat, title="Profil de la table contrat")
profil_contrat.to_file("profils/profil_contrat.html")

print("Pour la table option :")
profil_option = ProfileReport(option, title="Profil de la table option")
profil_option.to_file("profils/profil_option.html")

### Profil `resiliation_client`

In [ ]:
profil_client.to_notebook_iframe()  # Affiche le profil dans une iframe

La table `resiliation_client` contient des données personnelles sur les clients, à savoir leur genre, leur situation familliale (en couple, parent) et une indication sur leur âge. Il y a également une information sur leur ancienneté chez Bouygues Telecom (en mois je suppose).

Le profil donne un certain nombre d'informations interessantes :

* Il y a un total de 7043 observations et 7043 identifiants de clients différents. L'id_client est donc une clé primaire qui peut être utilisée pour joindre les tables.
* L'ancienneté a une distribution assez particulière :
  * Avec une forte concentration sur les clients avec une très faible ancienneté, avec 1062 des clients qui ont moins de 3 mois d'ancienneté, soit 15% du nombre total. On note que l'ancienneté diminue assez vite les premiers mois avant de se stabiliser.
  * Mais aussi sur les clients très anciens, avec 362 clients qui ont 72 mois d'anciennetés, soit 9% des clients. Il faut noter que 72 mois est le maximum recencé dans le dataset. On peut supposer que 72 correspond aux clients qui ont plus de 72 mois d'ancienneté. Cependant, on voit tout de même que 71 et 70 sont des valeurs très fréquentes.
  * Finalement, on peut noter des pics tous les 2 à 3 mois. Il pourrait être interressant de s'y intéresser, pour savoir si c'est notamment dû à des campagnes de marketing ou à des changements de tarifs par exemple.
* Il y a un total de 4305 observations manquantes soit 10.2% des données. Ces données manquantes sont réparties entre le genre des clients (où 10% des données sont manquantes) et leur situation parentale (où 51% des données sont manquantes). On peut voir que l'absence de la situation parentale n'implique pas l'absence du genre et inversement, les absences de ces variables ne sont pas corrélées.

Il peut être intéressant de noter la forte corrélation entre le fait d'être en couple et la situation parentale. En effet, le graphique suivant montre que parmi les clients en couple, la moitié sont parents, alors que parmi les clients qui ne sont pas en couple, la grosse majorité n'est pas parent.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")
%matplotlib inline

ax = sns.countplot(
    data=client.fillna({"parent": "NaN"}), x="couple", hue="parent", palette="Set2"
)
ax.set(
    xlabel="Couple",
    ylabel="Nombre de clients",
    title="Relation entre être en couple et être parent",
)
ax.legend(title="Parent")
plt.show()

De même, on voit clairement que les clients les plus anciens sont plus souvent en couple que les nouveaux clients. On peut donc penser que les clients célibataires sont plus enclins à résilier leur abonnement alors que les clients en couples le font moins souvent.

In [ ]:
ax = sns.kdeplot(
    data=client, x="anciennete", hue="couple", palette="Set2", cut=0, multiple="fill"
)
ax.set(
    xlabel="Ancienneté",
    ylabel="Pourcentage des clients",
    title="Proportion des clients en couple selon l'ancienneté",
)
plt.show()

### Profil `resiliation_contrat`

In [ ]:
profil_contrat.to_notebook_iframe()  # Affiche le profil dans une iframe

La table `resiliation_contrat` contient les données sur les contrats souscrits par les clients, à savoir le forfait, la durée du contrat (mensuel, annuel, biannuel), la méthode de paiement, la facture mensuelle moyenne, la facture totale et surtout si le client a résilé son abonnement ou non. 

Le profil donne un certain nombre d'informations interessantes :

* Comme dans la table sur les clients, l'id_client est une clé primaire. On vérifiera dans la suite que ce sont bien les mêmes clients qui sont présents dans les deux tables.
* On peut voir qu'il y a seulement 4 types de forfaits dans cette table, alors que la table sur les forfait en contient 5. Il y en a donc un qui n'est pas utilisé.
* Il y a moins d'observations manquantes que dans la table sur les clients, avec 9.7% des forfaits manquants.

In [ ]:
# On rajoute l'ancienneté à la table des contrats pour observer des corrélations
tmp1 = pd.merge(
    client[["id_client", "anciennete"]],
    contrat[
        [
            "id_client",
            "id_forfait",
            "facture_mensuelle_moyenne",
            "contrat",
            "resiliation",
        ]
    ],
    on="id_client",
)
tmp1 = pd.merge(tmp1, forfait, on="id_forfait", how="left")
tmp1.head()  # Table provisoire

Sur le graphique de gauche, on voit que l'ancienneté des clients a un effet très important sur la résiliation, en effet, les clients anciens ont moins tendance à résilier leur abonnement que les nouveaux clients. 

Dans un deuxième temps, le graphique de droite montre que, quelque soit le forfait, les clients qui ont résilier avaient une facture mensuelle plus faible que les clients qui n'ont pas résilier. Cela peut être dû à des promotions à durée fixe, et donc, une fois cette durée terminée, les clients ont tendance à résilier leur abonnement. Ce qui est en accordance avec le premier graphique.

Ensuite, on peut voir que les données avec le forfait manquant sont très différents des autres en terme de facture mensuelle. En effet, les factures se situent entre le forfait à 20Go et celui à 80Go. On note clairement que la facture mensuelle est très corrélée à la quantité de Go. On peut donc supposer que les forfait manquants correspondent à celui qui n'est pas utilisé et qui contient 50Go. Cette hypothèse sera retenue lors du traitement des valeurs manquantes.

Finalement, le dernier graphique montre que les longs contrats sont moins susceptibles d'être résilier que les contrats courts.

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(22, 5))
sns.kdeplot(
    data=tmp1, x="anciennete", hue="resiliation", multiple="stack", ax=ax1, cut=0
)
ax1.set(
    xlabel="Ancienneté",
    ylabel="Pourcentage des clients qui résilient",
    title="Résiliation selon l'ancienneté",
)
sns.boxplot(
    data=tmp1.fillna({"go_forfait": "NaN"}),
    x="go_forfait",
    y="facture_mensuelle_moyenne",
    ax=ax2,
    hue="resiliation",
)
ax2.set(
    xlabel="Forfait",
    ylabel="Facture mensuelle moyenne",
    title="Facture mensuelle moyenne par forfait",
)
sns.countplot(data=tmp1, x="contrat", hue="resiliation", ax=ax3)
ax3.set(
    xlabel="Durée du contrat",
    ylabel="Nombre de clients",
    title="Résiliation selon la durée du contrat",
)
plt.show()

### Profil `resiliation_option`

In [ ]:
profil_option.to_notebook_iframe()  # Affiche le profil dans une iframe

La table `resiliation_option` contient les données sur les options ajoutées aux forfaits par les clients, notamment des options de téléphonie, de télévision, de sécurité, de support ou de stream.

Les seules valeurs manquantes sont dans les options de streaming, qui sont corrélées, en effet, si une des options est manquantes, l'autre l'ait également.

On remarque que toutes les options sont très corrélées avec la variable `service_internet`. En effet, si le service internet n'est pas inclu dans le forfait, aucune options ne peut l'être.
Cependant, ce ne sont pas des variables redondantes, car il y a tout de même des options choisies par certains clients.

On peut faire la même remarque pour l'option téléphonie et les lignes multiples.

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(25, 5), sharey=True)
sns.countplot(data=option, x="service_internet", hue="option_securite", ax=axs[0])
axs[0].set(
    xlabel="Service internet",
    ylabel="Nombre de clients",
    title="Service internet selon l'option de sécurité",
)
sns.countplot(data=option, x="service_internet", hue="stream_TV", ax=axs[1])
axs[1].set(
    xlabel="Service internet",
    ylabel="Nombre de clients",
    title="Service internet selon l'option de stream TV",
)
sns.countplot(data=option, x="service_internet", hue="protection_terminal", ax=axs[2])
axs[2].set(
    xlabel="Service internet",
    ylabel="Nombre de clients",
    title="Service internet selon l'option protection terminal",
)
sns.countplot(
    data=option, x="option_service_telephone", hue="multiple_ligne", ax=axs[3]
)
axs[3].set(
    xlabel="Option service téléphonique",
    ylabel="Nombre de clients",
    title="Option service téléphonique selon l'option de multiple ligne",
)
plt.show()

Sur le graphique suivant, on constate encore que les clients avec un forfait manquant sont uniques, en effet, aucun n'ont l'option de service telephonique.
Cela renforce l'idée que les forfaits manquants sont dans une catégorie distincte, plutôt que des valeurs manquantes réparties entre les différents forfaits.

In [ ]:
tmp2 = pd.merge(
    contrat[["id_client", "id_forfait", "facture_mensuelle_moyenne"]],
    option,
    on="id_client",
)
ax = sns.countplot(
    data=tmp2.fillna({"id_forfait": "NaN"}),
    x="id_forfait",
    hue="option_service_telephone",
)
ax.set(
    xlabel="Forfait",
    ylabel="Nombre de clients",
    title="Nombre de clients par forfait en fonction de l'option de service",
)
plt.show()

## Préparation de données

Dans cette partie, nous allons travailler les données et les préparer pour qu'elles puissent être utilisées dans nos modèles.

Nous allons donc traiter le cas des valeurs manquantes, de la transformation des variables catégorielles en variables numériques.

Commençons par regrouper les données dans un dataframe nommé `df` en vérifiant que les id_client sont bien identiques.

In [ ]:
df = pd.merge(
    contrat,
    client,
    on="id_client",
    how="outer",
    validate="1:1",
    indicator="client_contrat",
)
df = pd.merge(
    df, option, on="id_client", how="outer", validate="1:1", indicator="client_option"
)
df = pd.merge(
    df, forfait, on="id_forfait", how="left", validate="m:1", indicator="client_forfait"
)
df.head()

In [ ]:
df["client_contrat"].value_counts()

In [ ]:
df["client_option"].value_counts()

In [ ]:
df["client_forfait"].value_counts()

On constate donc bien que les id_client sont des clés primaires et que les jointures se sont en *one to one*.

Pour les forfaits, comme mentionné précedemment, on voit qu'il y en a qui sont manquants et qu'un forfait n'est pas du tout présent. 

In [ ]:
profil_total = ProfileReport(df, title="Profil de la dataframe finale")
profil_total.to_file("profils/profil_total.html")

### Traitement des valeurs catégorielles

Pour commencer, nous allons évoquer les méthodes possibles pour traiter les valeurs catégorielles dans ces données.

In [ ]:
df.info()

In [ ]:
# from sklearn.dummy import DummyClassifier
# from sklearn.impute import MissingIndicator, SimpleImputer
# from sklearn.metrics import balanced_accuracy_score, f1_score
# from sklearn.model_selection import train_test_split
# from sklearn.pipeline import FeatureUnion, Pipeline, make_pipeline
# from sklearn.preprocessing import LabelBinarizer, LabelEncoder, StandardScaler
# from sklearn_pandas import DataFrameMapper

In [ ]:
# mapper = DataFrameMapper([("resiliation", LabelEncoder()), ([])])

In [ ]:
# import lightgbm as lgb

In [ ]:
# X, y = df.drop(["resiliation"], axis=1), df["resiliation"]
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, random_state=42
# )
# enc = LabelEncoder()
# train_data = lgb.Dataset(X_train, label=enc.fit_transform(y_train))